In [4]:
import pandas as pd
import numpy as np

# Load data
data = pd.read_csv("Project-II/Fraud.csv")
print(data.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [5]:
size,dim = data.shape
print(f"There are {size} online payments and {dim} variables.")
data = data.drop('isFlaggedFraud', axis=1)
print(data.head())

There are 6362620 online payments and 11 variables.
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  
0  M1979787155             0.0             0.0        0  
1  M2044282225             0.0             0.0        0  
2   C553264065             0.0             0.0        1  
3    C38997010         21182.0             0.0        1  
4  M1230701703             0.0             0.0        0  


In [ ]:
print(data.isnull().sum())
data = data.dropna()
print(data.isnull().sum())

In [ ]:
# Exploring transaction type
print(data.type.value_counts())

In [ ]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=transactions,hole = 0.5,
             title="Distribution of Transaction Type")
figure.show()

In [ ]:
data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})
#data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
print(data.head())

In [ ]:
# Checking correlation
correlation = data.iloc[:,[0,1,2,4,5,7,8,9]].corr()
print(correlation["isFraud"].sort_values(ascending=False))

In [ ]:
data["isFraud"] = data["isFraud"].map({0: "No", 1: "Yes"})

### Oline payments Fraud Dtection models

In [ ]:
from sklearn.model_selection import train_test_split

# splitting the data
X = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data[["isFraud"]])

### Decision trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# training a decision tree
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# Manually specify class weights (e.g., more weight to 'yes' class)
class_weights = {'No': 1, 'Yes': 10}  # Adjust the weight of the 'yes' class as needed
clf = DecisionTreeClassifier(random_state=42, class_weight=class_weights)
# try different hyper-parameters: criterion='gini', max_depth=None, min_samples_split=2

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# Manually specify class weights (e.g., more weight to 'yes' class)
class_weights = {'No': 1, 'Yes': 1000}  # Adjust the weight of the 'yes' class as needed
clf = DecisionTreeClassifier(random_state=42, class_weight=class_weights)

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

The sensitivity is improving, while the decrease in the accuracy is insignificant.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# Automatically balance class weights
clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model
clf = LogisticRegression(random_state=42, max_iter=1000)

# they require scaling

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForest model
clf = RandomForestClassifier(random_state=42)
# n_estimators, max_depth, n_jobs

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a GradientBoosting model
clf = GradientBoostingClassifier(random_state=42)

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Support Vector Machines

In [ ]:
from sklearn.svm import SVC

# Create a SVM model
clf = SVC(random_state=42)

# they require scaling

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Hyper parameter tuning

In [ ]:
# random forests
rf_model = RandomForestClassifier(
    n_estimators=100, max_depth=1, random_state=0, n_jobs=4)

# hyperparameter space
rf_param_grid = dict(
    n_estimators=[10, 20, 50, 100, 200, 500, 1000, 2000],
    max_depth=[1, 2, 3, 4],
)

# search
clf = GridSearchCV(rf_model,
                   rf_param_grid,
                   scoring=fnr_score,
                   cv=5)

search = clf.fit(X, y)

# best hyperparameters
search.best_params_